In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [5]:
BASE_DIR = os.getcwd()
ruta_csv = os.path.join(BASE_DIR, "..", "data", "diabetes.csv")
df = pd.read_csv(ruta_csv, sep=";", engine="python")
df.head()


,,,,,,,23,0
5,114,74,0,0,25,1,57,0
2,108,62,10,278,25,1,22,0
0,146,70,0,0,38,0,28,1
10,129,76,28,122,36,0,39,0
7,133,88,15,155,32,0,37,0


In [11]:
# Leer sin especificar separador
ruta_csv = os.path.join(BASE_DIR, "..", "data", "diabetes.csv")
with open(ruta_csv, "r") as f:
    lineas = f.readlines()

# Imprimir las primeras 3 líneas crudas
for i, linea in enumerate(lineas[:3]):
    print(f"Línea {i+1}: {linea}")


Línea 1: 23;0

Línea 2: 5;114;74;0;0;25;1;57;0

Línea 3: 2;108;62;10;278;25;1;22;0



In [17]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

#  Establecer ruta base
BASE_DIR = os.getcwd()
ruta_csv = os.path.join(BASE_DIR, "..", "data", "diabetes.csv")

#  Leer el CSV saltando la primera fila
df = pd.read_csv(ruta_csv, sep=";", skiprows=1, engine="python", header=None)

#  Asignar nombres correctos a las columnas
df.columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
              'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

#  Convertir todo a numérico por si acaso
df = df.apply(pd.to_numeric)

#  Mostrar primeras filas
print("🔍 Primeras filas del dataset limpio:")
display(df.head())


🔍 Primeras filas del dataset limpio:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,5,114,74,0,0,25,1,57,0
1,2,108,62,10,278,25,1,22,0
2,0,146,70,0,0,38,0,28,1
3,10,129,76,28,122,36,0,39,0
4,7,133,88,15,155,32,0,37,0


In [19]:
# Columnas que no deberían tener ceros
cols_to_fix = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

print(" Ceros por columna antes de limpieza:")
print(df[cols_to_fix].eq(0).sum())

# Reemplazar ceros con NaN y luego con la media
df[cols_to_fix] = df[cols_to_fix].replace(0, np.nan)
df.fillna(df.mean(), inplace=True)

print(" Ceros por columna después de limpieza:")
print(df[cols_to_fix].eq(0).sum())

# Validar dimensiones
print("📐 Tamaño del dataset final:", df.shape)


 Ceros por columna antes de limpieza:
Glucose            3
BloodPressure     23
SkinThickness    141
Insulin          228
BMI                6
dtype: int64
 Ceros por columna después de limpieza:
Glucose          0
BloodPressure    0
SkinThickness    0
Insulin          0
BMI              0
dtype: int64
📐 Tamaño del dataset final: (490, 9)


In [23]:
# Separar variables predictoras y target
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predecir
y_pred = model.predict(X_test)

# Evaluación
print("📉 Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\n📋 Reporte de clasificación:")
print(classification_report(y_test, y_pred))

# Guardar modelo
model_dir = os.path.join(BASE_DIR, "..", "models")
os.makedirs(model_dir, exist_ok=True)
ruta_modelo = os.path.join(model_dir, "model.pkl")
joblib.dump(model, ruta_modelo)

print(" Modelo guardado en:", ruta_modelo)


📉 Matriz de confusión:
[[55  7]
 [15 21]]

📋 Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.79      0.89      0.83        62
           1       0.75      0.58      0.66        36

    accuracy                           0.78        98
   macro avg       0.77      0.74      0.74        98
weighted avg       0.77      0.78      0.77        98

 Modelo guardado en: /Users/estebansaborio/Desktop/mlops_diabetes/notebooks/../models/model.pkl
